In [1]:
%cd "/content/drive/MyDrive/데이터사이언스"

/content/drive/MyDrive/데이터사이언스


In [2]:
!ls

 df_07_12.csv
 Untitled0.ipynb
 고장낸사람찾기.ipynb
'대여소별 이용정보(월별)_22.7-22.12.csv'
'대여소 정보(22.12월 기준).csv'
'대여이력 정보_2207.csv'
'대여이력 정보_2208.csv'
'대여이력 정보_2209.csv'
'대여이력 정보_2210.csv'
'대여이력 정보_2211.csv'
'대여이력 정보_2212.csv'
'서울시 공공자전거 고장신고 내역_22.07-12.csv'
'서울특별시 공공자전거 이용정보(일별)_2212.csv'
'신규가입자 정보(월별)_22.07~22.12.csv'


In [2]:
import pandas as pd

데이터프레임 불러오기

In [187]:
df = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2207.csv")
broken = pd.read_csv("서울시 공공자전거 고장신고 내역_22.07-12.csv")

날짜 연산을 위한 형변환

In [188]:
df = df[(df['대여일시'] != '0000-00-00 00:00:00') & (df['반납일시'] != '0000-00-00 00:00:00')]
df['대여일시'] = pd.to_datetime(df['대여일시'])
df['반납일시'] = pd.to_datetime(df['반납일시'])
df.drop(['대여거치대', '반납거치대', '대여대여소ID', '반납대여소ID'], axis=1, inplace=True)
df = df[(df['성별'] == 'M') | (df['성별'] == 'F')]
df = df[df['생년'] != '1901']
df = df[df['생년'] != '\\N']
df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')
df["반납대여소번호"] = df["반납대여소번호"].str.replace('N','-1')
df['대여요일'] = df['대여일시'].dt.day_name()

broken['등록일시'] = pd.to_datetime(broken['등록일시'])
broken = broken[pd.DatetimeIndex(broken['등록일시']).month == 7]


C:\Users\imihl\AppData\Local\Temp\ipykernel_12840\3382928667.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')


#고장낸사람찾기

중복 신고 날리기 위해, 가장 빠른 신고만 놔두고 다 드랍함

In [189]:
broken=broken.drop_duplicates('자전거번호')

대여이력과 고장신고내역을 합침

In [190]:
log=pd.merge(df,broken,on='자전거번호')

고장신고 전의 운행내역만 남김

In [191]:
log = log[log['대여일시']<=log['등록일시']]

실제 운행된 것으로 불 수 없는 레이블은 드랍함

In [192]:
log = log[log['대여 대여소명']!= log['반납대여소명']]#같은곳에서 반납된것 드랍
log = log[log['이용시간(분)']>2]#2분미만으로 탄 것 드랍 (이미 고장난 자전거를 대여했다가 반납한것으로 생각함)
log.shape

(173610, 15)

고장신고 등록일로부터 2일전까지의 내역만 남김 (고장에 유의미한 영향을 줄 수 있는 기간이라고 생각해서. 너무 과거를 볼 필요는 없으므로)

In [193]:
log = log[(log['등록일시']-log['반납일시']) <= pd.to_timedelta(2, unit = 'D')]
log.shape

(21577, 15)

In [194]:
log[log['자전거번호']=='SPB-56324'] #이 자전거는 3개 이용내역중 누가 고장낸걸로 볼 수 있을까? 마지막 이력이 유력함. 근데 첫번째분이 오래 타긴 했네..

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류,대여요일,등록일시,고장구분


In [195]:
# log2=log[log.duplicated(['자전거번호'], keep='last')]
log2 = log.drop_duplicates(['자전거번호'],keep='last')
print(log2.shape)

# log2.to_csv("brokenHistory.csv")

(3802, 15)


#고장안낸사람찾기(사용안함)

#수리다녀온자전거찾기



In [196]:
broken2 = pd.read_csv("서울시 공공자전거 고장신고 내역_22.07-12.csv")#헷갈려서 새로로드함.
df2 = pd.read_csv("서울특별시 공공자전거 대여이력 정보_2207.csv")

df2.drop(['대여거치대', '반납거치대', '대여대여소ID', '반납대여소ID'], axis=1, inplace=True)
df2 = df2[(df2['대여일시'] != '0000-00-00 00:00:00') & (df2['반납일시'] != '0000-00-00 00:00:00')]
df2['대여일시'] = pd.to_datetime(df2['대여일시'])
df2['반납일시'] = pd.to_datetime(df2['반납일시'])
df2['대여요일'] = df2['대여일시'].dt.day_name()

df2 = df2[(df2['성별'] == 'M') | (df2['성별'] == 'F')]
df2 = df2[df2['생년'] != '1901']
df2 = df2[df2['생년'] != '\\N']
df2["반납대여소번호"] = df2["반납대여소번호"].str.replace('\\','')
df2["반납대여소번호"] = df2["반납대여소번호"].str.replace('N','-1')

broken2['등록일시'] = pd.to_datetime(broken2['등록일시'])
broken2 = broken2[pd.DatetimeIndex(broken2['등록일시']).month == 7]

C:\Users\imihl\AppData\Local\Temp\ipykernel_12840\88983605.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2["반납대여소번호"] = df2["반납대여소번호"].str.replace('\\','')


In [197]:
broken2 = broken2.drop_duplicates('자전거번호',keep='last')#마지막 신고이력 남기기

good = pd.merge(df2, broken2, on='자전거번호')
good = good[good['대여일시']>good['등록일시']]
good_first = good.drop_duplicates('자전거번호',keep='first')

In [198]:
print(good.shape)#46411
print(good_first.shape)#1879

(13052, 15)
(1257, 15)


In [199]:
pd.merge(good, good_first, how='outer', indicator=True).query("_merge == 'left_only'").drop(columns=['_merge']).shape[0] + 1879 #디버깅용

13674

In [200]:
good_exceptfirst = pd.merge(good, good_first, how='outer', indicator=True).query("_merge == 'left_only'").drop(columns=['_merge'])
good_second = good_exceptfirst.drop_duplicates('자전거번호',keep='first')
good_second.head()

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류,대여요일,등록일시,고장구분
1,SPB-52539,2022-07-04 18:33:00,156,서울서부지방법원 앞,2022-07-04 18:52:00,147,마포역 4번출구 뒤,18,1587.86,1984,F,내국인,Monday,2022-07-02 23:59:00,페달
19,SPB-45293,2022-07-01 16:23:00,3865,이튼타워리버 2차,2022-07-01 16:27:00,3533,건대입구역 사거리(롯데백화점),3,450.00,2001,F,내국인,Friday,2022-07-01 08:14:00,페달
53,SPB-54712,2022-07-06 08:32:00,3408,종로타워빌딩,2022-07-06 08:36:00,320,을지로입구역 4번출구 앞,3,480.00,1982,M,내국인,Wednesday,2022-07-02 13:04:00,기타
57,SPB-49115,2022-07-07 12:40:00,4377,우영테크노센터 앞,2022-07-07 12:46:00,3541,커먼그라운드,5,1010.00,1994,M,내국인,Thursday,2022-07-06 18:50:00,기타
67,SPB-81336,2022-07-04 11:04:00,3536,중앙농협(자양동),2022-07-04 11:13:00,505,자양사거리 광진아크로텔 앞,8,910.00,1984,F,내국인,Monday,2022-07-01 08:31:00,타이어


In [201]:
good_firstAndSecond = pd.concat([good_first,good_second])
good_firstAndSecond.shape#3686

(2400, 15)

In [202]:
good_firstAndSecond = good_firstAndSecond[good_firstAndSecond['대여 대여소명']!= good_firstAndSecond['반납대여소명']]#같은곳에서 반납된것 드랍
good_firstAndSecond = good_firstAndSecond[good_firstAndSecond['이용시간(분)']>2]#2분미만으로 탄 것 드랍 (이미 고장난 자전거를 대여했다가 반납한것으로 생각함)

good_firstAndSecond.shape#3686

(2084, 15)

In [203]:
# good_firstAndSecond.to_csv("good.csv")

In [204]:
good_firstAndSecond['고장']=0 #라벨링
log2 = log2.sample(n=len(good_firstAndSecond))
log2['고장']=1
output = pd.concat([log2,good_firstAndSecond])
output.to_csv("training_data7.csv", encoding='cp949')

In [205]:
output.shape#(7735,19)

(4168, 16)

In [3]:
t7 = pd.read_csv("training_data7.csv")
t8 = pd.read_csv("training_data8.csv")
t9 = pd.read_csv("training_data9.csv")
t10 = pd.read_csv("training_data10.csv")
t11 = pd.read_csv("training_data11.csv")
t12 = pd.read_csv("training_data12.csv")

In [11]:
t = pd.concat([t7, t8, t9, t10, t11, t12])
t = t.drop(t.columns[0], axis=1)
t['대여일시'] = pd.to_datetime(t['대여일시'])
t = t.sort_values('대여일시')
print(t.head())

t.to_csv("training_data7~12.csv", encoding='cp949', index=False)

          자전거번호                대여일시  대여 대여소번호         대여 대여소명  \
1921  SPB-50867 2022-07-01 00:15:00       864  순천향대학병원(한남오거리)   
1403  SPB-40789 2022-07-01 00:23:00      5752   풍납백제문화공원 옆 인근   
1399  SPB-40134 2022-07-01 00:32:00      1339    삼성전자서비스 성북센터   
306   SPB-50932 2022-07-01 00:59:00       670    삼육서울병원 버스정류장   
481   SPB-55153 2022-07-01 01:00:00      1661         당현천근린공원   

                 반납일시  반납대여소번호             반납대여소명  이용시간(분)  이용거리(M)    생년 성별  \
1921  2022-07-01 0:32     4607  국립중앙박물관 -용산가족공원 앞       16  3280.00  1992  M   
1403  2022-07-01 0:31     1009     천호역4번출구(현대백화점)        7  1196.21  2001  M   
1399  2022-07-01 0:57     1306         한성대입구역2번출구       25  3409.46  1998  F   
306   2022-07-01 1:09      628           휘봉고등학교 앞        9  1103.01  1997  M   
481   2022-07-01 1:04     1656       중앙하이츠 아파트 입구        4   852.32  1997  M   

     이용자종류    대여요일              등록일시 고장구분  고장  
1921   내국인  Friday  2022-07-01 19:53   안장   1  
1403   내국인  Friday   2022-07-01 

In [8]:
len(t)

26414